In [61]:
import sqlite3
import csv
import os
import pandas as pd

In [ ]:
DB_NAME = "academic_ranking.db" # SQLite数据库文件名
CSV_DIR = "csv" # CSV文件目录
SCHEMA_FILE = "schema_sqlite.sql" # schema_sqlite.sql文件路径

## 初始化数据库

In [63]:
def init_database():
    conn = sqlite3.connect(DB_NAME) # 连接数据库
    with open(SCHEMA_FILE, 'r', encoding='utf-8') as f:
        conn.executescript(f.read()) # 执行schema_sqlite.sql文件中的SQL语句
    return conn

## 获取或创建学科ID

In [64]:
def get_or_create_field(cursor, field_name):
    cursor.execute("SELECT field_id FROM research_fields WHERE field_name = ?", (field_name,))
    result = cursor.fetchone()
    if result:
        return result[0]
    cursor.execute("INSERT INTO research_fields (field_name) VALUES (?)", (field_name,))
    return cursor.lastrowid

## 获取或创建机构ID

In [65]:
def get_or_create_institution(cursor, name, country):
    cursor.execute(
        "SELECT institution_id FROM institutions WHERE institution_name = ? AND country_region = ?",
        (name, country)
    )
    result = cursor.fetchone()
    if result:
        return result[0]
    # 判断是否为中国大陆，添加到机构表中
    is_chinese = 1 if country == "CHINA MAINLAND" else 0
    cursor.execute(
        "INSERT INTO institutions (institution_name, country_region, is_chinese_mainland) VALUES (?, ?, ?)",
        (name, country, is_chinese)
    )
    return cursor.lastrowid

## 导入单个CSV文件

In [66]:
def import_csv_file(cursor, csv_path, field_name):
    field_id = get_or_create_field(cursor, field_name) # 获取或创建学科ID
    with open(csv_path, 'r', encoding='utf-8', errors='ignore') as f:
        lines = f.readlines()
        reader = csv.DictReader(lines[1:]) # 从第3行开始读数据
        for row in reader:
            if not row.get('Institutions'): # 跳过空行或无效行
                continue
            # 清理数据（去除引号和空格）
            rank = (row.get('') or '').strip().strip('"')
            institution = (row.get('Institutions') or '').strip().strip('"')
            country = (row.get('Countries/Regions') or '').strip().strip('"')
            wos_docs = (row.get('Web of Science Documents') or '').strip().strip('"')
            cites = (row.get('Cites') or '').strip().strip('"')
            cites_per_paper = (row.get('Cites/Paper') or '').strip().strip('"')
            top_papers = (row.get('Top Papers') or '').strip().strip('"')

            # 获取或创建机构
            institution_id = get_or_create_institution(cursor, institution, country)
            
            # 插入排名数据
            cursor.execute("""
                INSERT INTO ranking_data 
                (field_id, institution_id, rank_position, wos_documents, cites, cites_per_paper, top_papers)
                VALUES (?, ?, ?, ?, ?, ?, ?)
            """, (
                field_id,
                institution_id,
                int(rank) if rank else 0, # 如果rank为空，则设置为0
                int(wos_docs) if wos_docs else 0, # 如果wos_docs为空，则设置为0
                int(cites) if cites else 0, # 如果cites为空，则设置为0
                float(cites_per_paper) if cites_per_paper else 0.00, # 如果cites_per_paper为空，则设置为0.00
                int(top_papers) if top_papers else 0 # 如果top_papers为空，则设置为0
            ))

In [67]:
# 初始化数据库
conn = init_database()
cursor = conn.cursor()

# 获取22个学科的CSV文件（排除0-ALL.csv）
csv_files = sorted([f for f in os.listdir(CSV_DIR) if f.endswith('.csv') and f != '0-ALL.csv'])

# 导入每个CSV文件
for csv_file in csv_files:
    # 从文件名提取学科名称
    field_name = csv_file.replace('.csv', '').split('-', 1)[1]
    csv_path = os.path.join(CSV_DIR, csv_file)
    
    print(f"正在导入: {field_name}...")
    import_csv_file(cursor, csv_path, field_name)
    conn.commit()

print(f"导入完成！")

正在导入: Agricultural Sciences...
正在导入: Immunology...
正在导入: Materials Science...
正在导入: Mathematics...
正在导入: Microbiology...
正在导入: Molecular Biology & Genetics...
正在导入: Multidisciplinary...
正在导入: Neuroscience & Behavior...
正在导入: Pharmacology & Toxicology...
正在导入: Physics...
正在导入: Plant & Animal Science...
正在导入: Biology & Biochemistry...
正在导入: Psychiatry Psychology...
正在导入: Social Sciences, General...
正在导入: Space Science...
正在导入: Chemistry...
正在导入: Clinical Medicine...
正在导入: Computer Science...
正在导入: Economics & Business...
正在导入: Engineering...
正在导入: Environment Ecology...
正在导入: Geosciences...
导入完成！


## 检查数据库内容

In [68]:
conn = sqlite3.connect("academic_ranking.db") # 连接

def sql2pd(sql):
    return pd.read_sql_query(sql, conn) # 将sql查询结果转换为DataFrame

# 查看所有学科
fields_df = sql2pd("SELECT * FROM research_fields ORDER BY field_id")
print(f"学科数量: {len(fields_df)}")
fields_df

学科数量: 22


,field_id,field_name
0,1,Agricultural Sciences
1,2,Immunology
2,3,Materials Science
3,4,Mathematics
4,5,Microbiology
5,6,Molecular Biology & Genetics
6,7,Multidisciplinary
7,8,Neuroscience & Behavior
8,9,Pharmacology & Toxicology
9,10,Physics


In [70]:
# 查看机构统计
institutions = sql2pd("""
    SELECT 
        COUNT(*) as total,
        COUNT(CASE WHEN is_chinese_mainland = 1 THEN 1 END) as chinese_mainland,
        COUNT(DISTINCT country_region) as countries
    FROM institutions
""")
print("机构统计:")
institutions

机构统计:


,total,chinese_mainland,countries
0,9990,859,140
